### Imports

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
import re
import datetime
import requests
from bs4 import BeautifulSoup
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Caminho da pasta "Twitch"
base_dir = Path(os.getcwd()).resolve().parent  # Ajuste se necessário

# Adicionar as pasta ao sys.path
data_dir = base_dir / 'data'
src_data_dir = base_dir / 'src' / 'data'
sys.path.append(str(data_dir))
sys.path.append(str(src_data_dir))

try:
    import API_Spotify as api
    import get_data
except ImportError:
    print("Erro: Não foi possível importar algum módulo. Verifique se o caminho está correto.")

C:\Users\diogo\Desktop\BDDA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get Data

In [2]:
# Diretório onde vão ficar os datasets
data_directory = data_dir / "raw"

def check_directory(data_directory):
    if os.path.exists(data_directory) and os.path.isdir(data_directory):
        if not os.listdir(data_directory):  # Verifica se a lista de ficheiros/diretórios está vazia
            print("A pasta está vazia.")
            return True
        else:
            print("A pasta não está vazia.")
            return False
    else:
        print("A pasta não existe ou não é um diretório.")
        return True

In [3]:
if check_directory(data_directory):
    # Lista de links dos datasets no Kaggle
    links = [
        "hedizekri/top-charts-artists-country",  # Países dos artistas
        "jackharding/spotify-artist-metadata-top-10k",  # Idade, género dos artistas
        "rodolfofigueroa/spotify-12m-songs",  # Spotify 12M Songs
        "estienneggx/spotify-unpopular-songs",  # Unpopular Songs
    ]

    # Executa o processo de download e conversão
    get_data.kaggleDownload(data_directory, links)

A pasta não está vazia.


## Artists

In [4]:
Artist_country =  pd.read_csv(data_directory / "Top_Charts_Artists_Country.csv")
Artist_metaData = pd.read_csv(data_directory / "top10k-spotify-artist-metadata.csv")
# Artist_tracks = pd.read_csv(data_directory /"tracks_features.csv")
Artist_country

,Artist,Country
0,Miley Cyrus,United States
1,FIFTY FIFTY,United Kingdom
2,ROSALÍA,Spain
3,PinkPantheress,United Kingdom
4,Metro Boomin,United States
...,...,...
960,Meck,United Kingdom
961,Matt Simons,United States
962,Hoodie Forester,Netherlands
963,NSB,United States


In [5]:
Artist_metaData

,Unnamed: 0,index,artist,gender,age,type,country,city_1,district_1,city_2,district_2,city_3,district_3
0,0,0,Drake,male,33,person,CA,NaN,NaN,Toronto,NaN,NaN,NaN
1,1,1,Post Malone,male,25,person,US,NaN,NaN,Syracuse,NaN,NaN,NaN
2,2,2,Ed Sheeran,male,29,person,GB,NaN,NaN,Halifax,NaN,NaN,NaN
3,3,3,J Balvin,male,35,person,CO,NaN,NaN,Medellín,NaN,NaN,NaN
4,4,4,Bad Bunny,male,26,person,PR,NaN,NaN,San Juan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,4991,si_el_bien,NaN,0,NaN,NaN,Chicago,NaN,NaN,NaN,NaN,NaN
9996,9996,4992,Antonello Venditti,male,71,person,IT,NaN,NaN,Rome,NaN,NaN,NaN
9997,9997,4993,Lea Salonga,female,49,person,PH,NaN,NaN,NaN,NaN,NaN,NaN
9998,9998,4994,Vertical Horizon,mixed,29,group,NaN,Boston,NaN,NaN,NaN,NaN,NaN


### Limpeza

In [6]:
def clean_text(text):
    if pd.isnull(text):  # Verifica se é nulo
        return text
    # Remove pontuação usando regex
    text = re.sub(r'[^\w\s]', '', text)
    # Converte para minúsculas
    text = text.title()
    # Remove espaços extras
    text = ' '.join(text.split())
    return text

# Colocar nomes uniformizados
Artist_metaData['artist_cleaned'] = Artist_metaData['artist'].apply(clean_text)
Artist_country['artist_cleaned'] = Artist_country['Artist'].apply(clean_text)

In [7]:
# Artistas que não estão presentes na DB secundária (vão ser apagados)
Artist_country[~Artist_country["artist_cleaned"].isin(Artist_metaData["artist_cleaned"])]

,Artist,Country,artist_cleaned
1,FIFTY FIFTY,United Kingdom,Fifty Fifty
3,PinkPantheress,United Kingdom,Pinkpantheress
5,David Kushner,United States,David Kushner
7,Rema,Nigeria,Rema
9,d4vd,United States,D4Vd
...,...,...,...
958,Big Brovaz,United Kingdom,Big Brovaz
960,Meck,United Kingdom,Meck
962,Hoodie Forester,Netherlands,Hoodie Forester
963,NSB,United States,Nsb


In [8]:
# Juntar as tabelas
if 'Artist_metaData' in locals() and 'Artist_country' in locals():
    # Faz o merge entre as duas tabelas com base na coluna 'artist_cleaned'
    merged_table = pd.merge(
        Artist_metaData,
        Artist_country,
        on='artist_cleaned',  # Coluna de ligação
        how='inner'           # Tipo de junção ('inner', 'left', 'right', 'outer')
    )
    del Artist_metaData, Artist_country

# Seleciona apenas as colunas que nos interessam
Artist_details = merged_table[['artist_cleaned', 'gender', 'age', 'Country']]
Artist_details = Artist_details.rename(columns={'artist_cleaned': 'artist_name', 'Country': 'country_born'})

# Queremos artistas singulares, não grupos
Artist_details = Artist_details[Artist_details['gender'] != 'mixed']

# A base de dados com a idade dos artistas foi publicada em 2021
## https://www.kaggle.com/datasets/jackharding/spotify-artist-metadata-top-10k/data
if 0 in Artist_details["age"].unique():
    Artist_details["age"] = Artist_details["age"] + datetime.datetime.now().year - 2021
Artist_details

,artist_name,gender,age,country_born
0,Drake,male,37,Canada
1,Post Malone,male,29,United States
2,Ed Sheeran,male,33,United Kingdom
3,J Balvin,male,39,Colombia
4,Bad Bunny,male,30,Puerto Rico
...,...,...,...,...
651,Mo,NaN,4,United Kingdom
660,Sisqo,male,45,United States
664,Waka Flocka Flame,male,38,United States
665,Lou Bega,male,49,Germany


In [9]:
Artist_details[Artist_details.isnull().any(axis=1)]

,artist_name,gender,age,country_born
53,Annemarie,NaN,4,France
314,Bizarrap,NaN,26,Argentina
357,Big Shaq,NaN,32,United Kingdom
388,Giveon,NaN,4,United States
399,Nsg,NaN,4,Indonesia
424,Mo,NaN,4,United Kingdom
508,Chip,NaN,4,United Kingdom
587,Lyodra,NaN,4,Indonesia
602,Diskoria,NaN,4,Indonesia
651,Mo,NaN,4,United Kingdom


In [10]:
# Definir manualmente os valores da coluna 'gender' para alguns artistas
Artist_details.loc[Artist_details['artist_name'] == 'Annemarie', 'gender'] = 'female'
Artist_details.loc[Artist_details['artist_name'] == 'Bizarrap', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Big Shaq', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Giveon', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Chip', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Lyodra', 'gender'] = 'Female'

# Apagar colunas com valores nulos
Artist_details = Artist_details.dropna(axis=0)

In [11]:
# Definir manualmente os valores da coluna 'age' para alguns artistas
Artist_details.loc[Artist_details['artist_name'] == 'Annemarie', 'age'] = 33 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Bizarrap', 'age'] = 26 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Big Shaq', 'age'] = 33 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Giveon', 'age'] = 29 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Chip', 'age'] = 34 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Lyodra', 'age'] = 21 + (2025 - datetime.datetime.now().year)
Artist_details.loc[Artist_details['artist_name'] == 'Joel Adams', 'age'] = 28 + (2025 - datetime.datetime.now().year)

In [12]:
incorretAge = Artist_details[Artist_details["age"] == datetime.datetime.now().year - 2021][["artist_name", "age"]]
incorretAge

,artist_name,age
50,Manuel Turizo,4
66,Offset,4
85,Paulo Londra,4
93,Tones And I,4
109,Rauw Alejandro,4
124,Lil Pump,4
157,Meduza,4
174,Nio Garcia,4
183,Dynoro,4
188,Saint Jhn,4


In [13]:
# Atualizar as idades na tabela
incorretAge = api.update_artist_ages(incorretAge)

# Atualizar a coluna 'age' na tabela principal
Artist_details = Artist_details.merge(
    incorretAge,  # Tabela secundária
    on='artist_name',  # Coluna de ligação
    how='left',  # Faz uma junção à esquerda
    suffixes=('', '_secondary')  # Adiciona sufixos para diferenciar colunas
)

# Preencher os valores de 'age' com os dados da tabela secundária, se disponíveis
Artist_details['age'] = Artist_details['age_secondary'].combine_first(Artist_details['age'])

# Remover a coluna auxiliar
Artist_details = Artist_details.drop(columns=['age_secondary'])

Artist_details.dropna(inplace=True)

Artist_details

,artist_name,gender,age,country_born
0,Drake,male,37.0,Canada
1,Post Malone,male,29.0,United States
2,Ed Sheeran,male,33.0,United Kingdom
3,J Balvin,male,39.0,Colombia
4,Bad Bunny,male,30.0,Puerto Rico
...,...,...,...,...
487,Miguel,male,54.0,United States
488,Sisqo,male,45.0,United States
489,Waka Flocka Flame,male,38.0,United States
490,Lou Bega,male,49.0,Germany


In [14]:
Artist_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   492 non-null    object 
 1   gender        492 non-null    object 
 2   age           492 non-null    float64
 3   country_born  492 non-null    object 
dtypes: float64(1), object(3)
memory usage: 15.5+ KB


In [15]:
Artist_details.describe()

,age
count,492.000000
mean,36.215447
std,11.861142
min,4.000000
25%,30.000000
50%,36.000000
75%,43.000000
max,96.000000


In [16]:
nome_correcoes = {
    "J Cole": "J. Cole",
    "Aap Rocky": "A$AP Rocky",
    "Gigi Dagostino": "Gigi D'Agostino",
    "Machine Gun Kelly": "mgk",
    "Tyler The Creator": "Tyler, The Creator",
    "Dram": "DRAMA",
    "Mr Probz": "Mr. Probz",
    "Russ": "Russ Millions",
    "Neyo": "Ne-Yo",
    "Marc E Bassy": "Marc E. Bassy",
    "Rob Tone": "Rob $tone",
    "Ruth B": "Ruth B.",
    "Mk": "Mk.gee",
    "Ot Genasis": "O.T. Genasis",
    "The Notorious Big": "The Notorious B.I.G.",
    "Eo": "ÉoDan",
    "Dr Dre": "Dr. Dre",
    "Mia": "M.I.A.",
    "Mary J Blige": "Mary J. Blige",
    "Jojo": "Jojo's ASMR",
    "pnk": "pnkcité",
    "Geazy": "G-Eazy",
    "Marina": "MARINA",
    "Thedream":"The-Dream",
    "Ti":"T.I.",
    "Aap Ferg":"A$AP Ferg",
    "Annemarie":"Anne-Marie",
    "Blackbear": "blackbear",
    "Iann Dior": "iann dior",
}

# Atualiza os nomes no DataFrame
Artist_details["artist_name"] = Artist_details["artist_name"].replace(nome_correcoes)

### Api

In [17]:
# Demora, aproximadamente 0.26626 segundos por artista
# Primeiro é necessário rodar o ficheiroi API_Spotify.py no src/data
# Cuidado, é necessário apagar o ficheiro .cache deste diretório

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="ca14c7679c394052af165d2168f3361f",
    client_secret="8b250ef320964c2a9197f5453f3c449d",
    redirect_uri="https://localhost:1234/callback",
    scope="user-library-read",  # Definindo a scope necessária
    requests_timeout=15))

# Problemas:
## PlayBoi Carti
api.get_artist_details(Artist_details, 'artist_name', sp)
Artist_details[Artist_details.isnull().any(axis=1)]

Nome fornecido: Mø, Nome retornado pela API: Morad
Nome fornecido: Pnk, Nome retornado pela API: pnkcité
Nome fornecido: Miguel, Nome retornado pela API: Miguel Luz
Nome fornecido: Sebastian Ingrosso, Nome retornado pela API: Axwell /\ Ingrosso
Nome fornecido: Nelly, Nome retornado pela API: Nelly Furtado
Nome fornecido: Philip George, Nome retornado pela API: Philippe Entremont
Nome fornecido: Bob, Nome retornado pela API: Bob Marley
Nome fornecido: Bob, Nome retornado pela API: Bob Marley
Nome fornecido: Miguel, Nome retornado pela API: Miguel Luz


,artist_name,gender,age,country_born,artist_id,followers,popularity,genres,image_url
71,Mø,female,35.0,Denmark,Morad,NaN,NaN,None,None
166,Yg,male,34.0,United States,7iqXu8imQ1rePIk5Nx1ctV,5.0,0.0,,None
175,Pnk,female,44.0,United States,pnkcité,NaN,NaN,None,None
196,Miguel,male,54.0,United States,Miguel Luz,NaN,NaN,None,None
311,Sebastian Ingrosso,male,41.0,Sweden,Axwell /\ Ingrosso,NaN,NaN,None,None
320,Zhu,female,4.0,China,2QHqarwkjOlx0HtV7KyEx9,9.0,0.0,,None
321,Nelly,male,49.0,United States,Nelly Furtado,NaN,NaN,None,None
362,Tpain,male,38.0,United States,5Daf0heTiEznki7hU5GerJ,4.0,0.0,,None
372,Philip George,male,31.0,United Kingdom,Philippe Entremont,NaN,NaN,None,None
398,Bob,male,40.0,United States,Bob Marley,NaN,NaN,None,None


In [18]:
Artist_details.dropna(inplace=True)
Artist_details.dropna(subset=['genres'], inplace=True)
Artist_details.drop_duplicates(inplace=True)

processed_data = data_dir / 'processed'
if not os.path.exists(processed_data):
    os.makedirs(processed_data)
Artist_details.to_csv(processed_data / 'artist_details.csv', index=False)
Artist_details

,artist_name,gender,age,country_born,artist_id,followers,popularity,genres,image_url
0,Drake,male,37.0,Canada,3TVXtAsR1Inumwj472S9r4,94896586.0,97.0,"canadian hip hop, canadian pop, hip hop, pop r...",https://i.scdn.co/image/ab6761610000e5eb429338...
1,Post Malone,male,29.0,United States,246dkjvS1zLTtiykXe5h60,46197705.0,91.0,"dfw rap, melodic rap, pop, rap",https://i.scdn.co/image/ab6761610000e5ebe17c0a...
2,Ed Sheeran,male,33.0,United Kingdom,6eUKZXaKkcviH0Ku9w2n3V,118283385.0,90.0,"pop, singer-songwriter pop, uk pop",https://i.scdn.co/image/ab6761610000e5eb784daf...
3,J Balvin,male,39.0,Colombia,1vyhD5VmyZ7KMfW5gqLgo5,37649707.0,88.0,"reggaeton, reggaeton colombiano, trap latino, ...",https://i.scdn.co/image/ab6761610000e5ebdea087...
4,Bad Bunny,male,30.0,Puerto Rico,4q3ewBCX7sLwd24euuV69X,88433615.0,96.0,"reggaeton, trap latino, urbano latino",https://i.scdn.co/image/ab6761610000e5eb744a42...
...,...,...,...,...,...,...,...,...,...
485,Lenny Kravitz,male,60.0,United States,5gznATMVO85ZcLTkE9ULU7,3427889.0,74.0,"permanent wave, rock",https://i.scdn.co/image/ab6761610000e5ebe0cf05...
486,Daniel Bedingfield,male,44.0,United Kingdom,11hIqBsGRPztdjBHCSLClX,225333.0,55.0,"europop, talent show",https://i.scdn.co/image/411b4430f842fca92fedf7...
488,Sisqo,male,45.0,United States,6x9QLdzo6eBZxJ1bHsDkjg,1099676.0,60.0,"contemporary r&b, hip pop, r&b, urban contempo...",https://i.scdn.co/image/65c4f2d897d65c2236ddc7...
489,Waka Flocka Flame,male,38.0,United States,6f4XkbvYlXMH0QgVRzW0sM,2744219.0,68.0,"atl hip hop, dirty south rap, pop rap, rap, so...",https://i.scdn.co/image/ab6761610000e5eb098d90...


## Popular Individual Tracks

In [19]:
tracks = Artist_details[["artist_id", "artist_name"]]
tracks

,artist_id,artist_name
0,3TVXtAsR1Inumwj472S9r4,Drake
1,246dkjvS1zLTtiykXe5h60,Post Malone
2,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran
3,1vyhD5VmyZ7KMfW5gqLgo5,J Balvin
4,4q3ewBCX7sLwd24euuV69X,Bad Bunny
...,...,...
485,5gznATMVO85ZcLTkE9ULU7,Lenny Kravitz
486,11hIqBsGRPztdjBHCSLClX,Daniel Bedingfield
488,6x9QLdzo6eBZxJ1bHsDkjg,Sisqo
489,6f4XkbvYlXMH0QgVRzW0sM,Waka Flocka Flame


In [20]:
teste = tracks.head()
teste

,artist_id,artist_name
0,3TVXtAsR1Inumwj472S9r4,Drake
1,246dkjvS1zLTtiykXe5h60,Post Malone
2,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran
3,1vyhD5VmyZ7KMfW5gqLgo5,J Balvin
4,4q3ewBCX7sLwd24euuV69X,Bad Bunny


In [21]:
tracks_with_features = api.get_artist_tracks(teste, sp)

Processando artista Drake (1/5)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1zi7xx7UVEFkmKfv06H8x0 with Params: {} returned 403 due to None


Erro: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1zi7xx7UVEFkmKfv06H8x0:
 None, reason: None. Tentativa 1 de 3


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1zi7xx7UVEFkmKfv06H8x0 with Params: {} returned 403 due to None


Erro: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1zi7xx7UVEFkmKfv06H8x0:
 None, reason: None. Tentativa 2 de 3


...

Falha após múltiplas tentativas.
Erro: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1IHWl5LamUGEuP4ozKQSXZ:
 None, reason: None. Tentativa 1 de 3


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1IHWl5LamUGEuP4ozKQSXZ with Params: {} returned 403 due to None


Erro: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1IHWl5LamUGEuP4ozKQSXZ:
 None, reason: None. Tentativa 2 de 3


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1IHWl5LamUGEuP4ozKQSXZ with Params: {} returned 403 due to None


Erro: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1IHWl5LamUGEuP4ozKQSXZ:
 None, reason: None. Tentativa 3 de 3
Falha após múltiplas tentativas.
